In [1]:
import pandas as pd
import camelot # um Tabellen im PDFs zu erkennen
import PyPDF2 # um Text aus PDFs zu lesen
import re

In [2]:
file = "Sessionsprotokolle/SR_5102_2003.pdf"

In [3]:
dokument = PyPDF2.PdfReader(file)

In [4]:
regex_gnummer = r"(\d\d\.\S+)"
regex_datum = r"\d\d\.\d\d\.\d\d\d\d \d\d\:\d\d\:\d\d"

In [5]:
seitenliste = []

laenge = len(dokument.pages)

for seite in range(laenge):
    inhalt = dokument.pages[seite].extract_text()
    if inhalt[:8] == "Geschäft":
        nummer = seite
        seitenliste.append(nummer)

In [6]:
master_df_list = []

for seite in seitenliste:
    inhalt = dokument.pages[seite].extract_text()
    datenliste = inhalt.split("\n")
    
    for zeile in range(len(datenliste)):
        if "Abstimmung vom" in datenliste[zeile]:
        
            datum = re.findall(regex_datum, datenliste[zeile])[0]
            
            print(datum)
    
    for zeile in range(len(datenliste)):
        if "Geschäft" in datenliste[zeile]:
            geschäftszeile = datenliste[(zeile+1)]
            
            gnummer = (re.findall(regex_gnummer, geschäftszeile))[0]
            geschäft = geschäftszeile.strip(str(gnummer)+" ")
            
            print(gnummer)
            print(geschäft)
            print(" ")
    
            
    tables = camelot.read_pdf(file, flavor='lattice', pages = str(seite))
    
    try: 
        for table in range(0,2):
            abstimmungs_df = tables[table].df
    
            zeilen = len(abstimmungs_df.index)

            for zeile in range(0,zeilen):    
                name = abstimmungs_df.loc[zeile, 1] + " " +abstimmungs_df.loc[zeile, 0]
                kanton = abstimmungs_df.loc[zeile, 3]
                verhalten = abstimmungs_df.loc[zeile, 2]

                mini_dict = {
                    "Datum" : datum,
                    "Nummer" : gnummer,
                    "Geschäft" : geschäft,
                    "Politiker" : name,
                    "Kanton" : kanton,
                    "Verhalten" : verhalten}
    
                master_df_list.append(mini_dict)
        
    #hier kommt der unten entwickelte Alternativweg rein, bei dem Text ausgelesen wird, weil Tabelle nicht erkannt wird:
    except:
        # So holen wir die richtigen 24 Elemente aus der Tabelle
        tabellenanfang = datenliste.index(' ')+1
        tabelle = datenliste[tabellenanfang:(tabellenanfang+24)]
        
        # So flicken wir die Gmür-Schnönenberger:
        tabelle.remove("Gmür-")
        tabelle = [w.replace('SchönenbergerAndrea', 'Gmür-Schönenberger Andrea') for w in tabelle]
        
        # und jetzt trennen wir jede Zeile beim ersten Auftauchen eines Kantons (jenes mit Leerschlag)
        tabelle_getrennt = []
        kantonsliste = ["VD", "TG", "TI", "AG", "JU", "GL", "UR", "SO", "NE", "SZ", "GR", "BE", "SG", "ZH", "LU", "SH", "ZG", "GE", "VS", "FR", "OW", "BL", "AI", "BS", "AR", "NW"]

        for paar in tabelle:
            for kanton in kantonsliste:
                trenner = kanton + " "
                if trenner in paar:
                    trennung = paar.split(trenner)
            
                    # (dann müssen wir den bei wplit verloren gegangenen Kanton, den Trenner, wieder hinzufügen)
                    trennung[0] = trennung[0]+kanton
            
                    tabelle_getrennt.append(trennung[0])
                    tabelle_getrennt.append(trennung[1])
                    
        for eintrag in tabelle_getrennt:
            eintrag_neu = eintrag.rsplit(" ", 2)
            
            name = eintrag_neu[0]
            kanton = eintrag_neu[2]
            verhalten = eintrag_neu[1]
            
            mini_dict = {
            "Datum" : datum,
            "Nummer" : gnummer,
            "Geschäft" : geschäft,
            "Politiker" : name,
            "Kanton" : kanton,
            "Verhalten" : verhalten}
    
            master_df_list.append(mini_dict)

09.03.2020 18:19:01
18.071
Terrorismus und organisierte Kriminalität. Übereinkommen des Europarates
 


/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/camelot/parsers/lattice.py:417: UserWarning: No tables found on page-200
  "No tables found on {}".format(os.path.basename(self.rootname))


04.03.2020 11:37:37
18.076
Bundesgesetz über das Internationale Privatrecht. 12. Kapitel: Internationale Schiedsgerichtsbarkeit
 
02.03.2020 17:45:41
18.092
Erwerbsersatzgesetz. Mutterschaftsentschädigung bei längerem Spitalaufenthalt des Neugeborenen
 
02.03.2020 17:02:00
19.030
Tierseuchengesetz. Änderung
 
09.03.2020 19:40:30
19.032-1
Polizeiliche Massnahmen zur Bekämpfung von Terrorismus. Bundesgesetz
 
03.03.2020 09:39:02
19.033
Einführung des automatischen Informationsaustauschs über Finanzkonten mit weiteren Partnerstaaten ab
 
02.03.2020 17:21:13
19.047
Übereinkommen des Europarats gegen den Handel mit menschlichen Organen. Genehmigung und
 
10.03.2020 10:18:42
19.051-1
Überbrückungsleistung für ältere Arbeitslose. Bundesgesetz
 
04.03.2020 08:33:21
19.471
Pa.Iv. Comte. Opfer fürsorgerischer Zwangsmassnahmen. Fristverlängerung
 
04.03.2020 08:35:27
19.471
Pa.Iv. Comte. Opfer fürsorgerischer Zwangsmassnahmen. Fristverlängerung
 
11.03.2020 09:12:04
20.008-1
Aussenwirtschaftspoli

In [197]:
master_df = pd.DataFrame(master_df_list)
master_df

,Datum,Nummer,Geschäft,Politiker,Kanton,Verhalten
0,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Bauer Philippe,NE,+
1,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Kuprecht Alex,SZ,+
2,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Baume-Schneider Elisabeth,JU,+
3,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Levrat Christian,FR,+
4,09.03.2020 18:19:01,18.071,Terrorismus und organisierte Kriminalität. Übe...,Bischof Pirmin,SO,+
...,...,...,...,...,...,...
639,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Peter Hegglin,ZG,0
640,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Eva Herzog,BS,+
641,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Daniel Jositsch,ZH,+
642,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Charles Juillard,JU,+


## Paare pro Geschäft finden und auswerten

In [ ]:
# zuerst müssen wir den Dataframe nach geschäft gruppieren:

In [166]:
# Das hier generiert uns eine Liste mit allen Geschäften:
master_df["Datum"].value_counts().index.to_list()

['09.03.2020 18:19:01',
 '04.03.2020 11:37:37',
 '02.03.2020 17:45:41',
 '02.03.2020 17:02:00',
 '09.03.2020 19:40:30',
 '03.03.2020 09:39:02',
 '02.03.2020 17:21:13',
 '10.03.2020 10:18:42',
 '04.03.2020 08:33:21',
 '04.03.2020 08:35:27',
 '11.03.2020 09:12:04',
 '11.03.2020 09:12:50',
 '11.03.2020 09:13:36',
 '11.03.2020 09:14:16']

In [280]:
# So steuern wir ein einzelnes Geschäft an
master_df[master_df["Datum"] == "11.03.2020 09:14:16"]

,Datum,Nummer,Geschäft,Politiker,Kanton,Verhalten
598,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Alex Kuprecht,SZ,+
599,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Christian Levrat,FR,0
600,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Marianne Maret,VS,+
601,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Lisa Mazzone,GE,+
602,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Matthias Michel,ZG,+
603,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Thomas Minder,SH,+
604,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Damian Müller,LU,+
605,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Ruedi Noser,ZH,+
606,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Paul Rechsteiner,SG,0
607,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Othmar Reichmuth,SZ,+


In [ ]:
geschaeft_df = master_df[master_df["Datum"] == "11.03.2020 09:12:50"]

In [142]:
for kanton in kantonsliste:
    kanton_df = geschaeft_df[geschaeft_df["Kanton"] == kanton]
    kanton_df

In [150]:
kanton_df = geschaeft_df[geschaeft_df["Kanton"] == "JU"]
kanton_df

,Datum,Nummer,Geschäft,Politiker,Kanton,Verhalten
508,11.03.2020 09:12:50,20.008-2,Aussenwirtschaftspolitik 2019. Bericht,Baume-Schneider Elisabeth,JU,+
548,11.03.2020 09:12:50,20.008-2,Aussenwirtschaftspolitik 2019. Bericht,Juillard Charles,JU,+


In [191]:
standesstimme = kanton_df["Verhalten"].value_counts().to_dict()
standesstimme

{'+': 2}

TypeError: '>=' not supported between instances of 'dict' and 'int'

In [154]:
kanton_df.index.to_list()

[508, 548]

In [155]:
# hiermit suchen wir nach den jeweiligen Positionen der gleichen Kantone
position_gleiche = kanton_df.index.to_list()

for zeile in position_gleiche:
    
    #hier erledigen wir jene Kantone, die nur einen EIntrag haben
    if len(kanton_df) <= 1:
        kanton_df.loc[zeile, "Standesstimme"] = "Halbkanton"
     
    #hier erledigen wir jene, die aus verschiedenen Gründen nicht vollzählig waren
    elif "E" in standesstimme:
        kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
    elif "P" in standesstimme:
        kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
    elif "0" in standesstimme:
        kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
   
    #hier kommen jene, die gelich abgstimmt haben
    elif standesstimme["+"] == 2:
        kanton_df.loc[zeile, "Standesstimme"] = "beide dafür"
    elif standesstimme["-"] == 2:
        kanton_df.loc[zeile, "Standesstimme"] = "beide dagegen"
    elif standesstimme["="] == 2:
        kanton_df.loc[zeile, "Standesstimme"] = "beide enthalten"
        
    #und schliesslich die anderen
    else:
        kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
        

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [243]:
kanton_df

,Datum,Nummer,Geschäft,Politiker,Kanton,Verhalten,Standesstimme
616,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Hans Wicki,NW,+,Halbkanton


## Erster Versuch, alles in einen Loop zu hängen

In [266]:
# Das hier generiert uns eine Liste mit allen Geschäften und allen Kantonen:
geschaeftsliste = master_df["Datum"].value_counts().index.to_list()
kantonsliste = ["VD", "TG", "TI", "AG", "JU", "GL", "UR", "SO", "NE", "SZ", "GR", "BE", "SG", "ZH", "LU", "SH", "ZG", "GE", "VS", "FR", "OW", "BL", "AI", "BS", "AR", "NW"]

In [267]:
# So schalte ich die Fehlermeldung aus
pd.options.mode.chained_assignment = None  # default='warn'

In [276]:
df_komplett = pd.DataFrame()

for geschaeft in geschaeftsliste:
    geschaeft_df = master_df[master_df["Datum"] == geschaeft]
    
    for kanton in kantonsliste:
        kanton_df = geschaeft_df[geschaeft_df["Kanton"] == kanton]
        standesstimme = kanton_df["Verhalten"].value_counts().to_dict()
        position_gleiche = kanton_df.index.to_list()

        for zeile in position_gleiche:
    
            #hier erledigen wir jene Kantone, die nur einen EIntrag haben
            if len(kanton_df) <= 1:
                kanton_df.loc[zeile, "Standesstimme"] = "Halbkanton"
     
            #hier erledigen wir jene, die aus verschiedenen Gründen nicht vollzählig waren
            elif "E" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
            elif "P" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
            elif "0" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
   
            #hier kommen jene, die gelich abgstimmt haben
            elif "+" in standesstimme:
                if standesstimme["+"] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide dafür"
                if standesstimme["+"] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
            elif "-" in standesstimme:
                if standesstimme["-"] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide dagegen"
                if standesstimme["-"] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
            elif "=" in standesstimme:
                if standesstimme["="] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide enthalten"
                if standesstimme["="] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
        
            #und schliesslich die anderen
            else:
                kanton_df.loc[zeile, "Standesstimme"] = "leer"
                
        df_komplett = df_komplett.append(kanton_df)

In [278]:
df_komplett.tail(50)

,Datum,Nummer,Geschäft,Politiker,Kanton,Verhalten,Standesstimme
585,11.03.2020 09:13:36,20.008-3,Aussenwirtschaftspolitik 2019. Bericht,Daniel Fässler,AI,+,Halbkanton
594,11.03.2020 09:13:36,20.008-3,Aussenwirtschaftspolitik 2019. Bericht,Eva Herzog,BS,+,Halbkanton
580,11.03.2020 09:13:36,20.008-3,Aussenwirtschaftspolitik 2019. Bericht,Andrea Caroni,AR,0,Halbkanton
570,11.03.2020 09:13:36,20.008-3,Aussenwirtschaftspolitik 2019. Bericht,Hans Wicki,NW,+,Halbkanton
614,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Adèle Thorens Goumaz,VD,+,beide dafür
632,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Olivier Français,VD,+,beide dafür
612,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Jakob Stark,TG,+,beide dafür
637,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Brigitte Häberli-Koller,TG,+,beide dafür
625,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Marina Carobbio Guscetti,TI,+,nicht vollzählig
627,11.03.2020 09:14:16,20.008-4,Aussenwirtschaftspolitik 2019. Bericht,Marco Chiesa,TI,0,nicht vollzählig


In [279]:
len(df_komplett)

644